In [1]:
import pandas as pd

In [2]:
application_df: pd.DataFrame = pd.read_csv("./data/application_rec.csv")
credit_df: pd.DataFrame = pd.read_csv("./data/credit_rec.csv")

In [6]:
application_df.head()

,Unnamed: 0,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2
1,2,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2
2,3,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2
3,4,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1
4,5,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1


- `ID`: Identyfikator klienta
- `STATUS`: Status
- `START_DATE`: Data uruchomienia karty kredytowej
- `SNAPSHOT_MONTH`: Miesiąc snapshotu

In [4]:
credit_df.head()

,Unnamed: 0,X,ID,STATUS,START_DATE,SNAPSHOT_MONTH
0,92833,92953,5008804,X,2021-09-01,2021-09-01
1,92834,92952,5008804,0,2021-09-01,2021-10-01
2,92835,92951,5008804,1,2021-09-01,2021-11-01
3,92836,92950,5008804,C,2021-09-01,2021-12-01
4,92837,92949,5008804,C,2021-09-01,2022-01-01


Kolumna `STATUS`:
- 0: brak opoźnienia lub opóźnienie <30 dni
- 1: opóźnienie w spłacie 30-59 dni
- 2: opóźnienie w spłacie 60-89 dni
- 3: opóźnienie w spłacie 90-119 dni
- 4: opóźnienie w spłacie 120-149 dni
- 5: opóźnienie w spłacie 150+ dni
- C: karta zamknięta
- X: brak zadużenia w danym miesiącu

In [ ]:
credit_df["STATUS"].value_counts()

STATUS
C    329461
0    290578
X    145947
1      8734
5      1526
2       801
3       286
4       214
Name: count, dtype: int64

In [8]:
application_df.info(), credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36448 entries, 0 to 36447
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           36448 non-null  int64  
 1   ID                   36448 non-null  int64  
 2   CODE_GENDER          36448 non-null  object 
 3   FLAG_OWN_CAR         36448 non-null  object 
 4   FLAG_OWN_REALTY      36448 non-null  object 
 5   CNT_CHILDREN         36448 non-null  int64  
 6   AMT_INCOME_TOTAL     36448 non-null  float64
 7   NAME_INCOME_TYPE     36448 non-null  object 
 8   NAME_EDUCATION_TYPE  36448 non-null  object 
 9   NAME_FAMILY_STATUS   36448 non-null  object 
 10  NAME_HOUSING_TYPE    36448 non-null  object 
 11  DAYS_BIRTH           36448 non-null  int64  
 12  DAYS_EMPLOYED        36448 non-null  int64  
 13  FLAG_MOBIL           36448 non-null  int64  
 14  FLAG_WORK_PHONE      36448 non-null  int64  
 15  FLAG_PHONE           36448 non-null 

(None, None)

Definicja `BAD_1`: 90+ in 12 MoB

Definicja `BAD_2`: 30+ in 5 MoB

Definicja `BAD_3`: 60+ in 18 MoB

In [ ]:
credit_df[
    (credit_df["SNAPSHOT_MONTH"] <= "2018-10-01")
    & (credit_df["START_DATE"] == "2018-05-01")
    & (~credit_df["STATUS"].isin(["0", "C", "X"]))
]["ID"].nunique()

28

In [ ]:
credit_df["START_DATE"] = pd.to_datetime(credit_df["START_DATE"])
credit_df["SNAPSHOT_MONTH"] = pd.to_datetime(credit_df["SNAPSHOT_MONTH"])

credit_df["MoB"] = (
    credit_df["SNAPSHOT_MONTH"].dt.year - credit_df["START_DATE"].dt.year
) * 12 + (credit_df["SNAPSHOT_MONTH"].dt.month - credit_df["START_DATE"].dt.month)
credit_df.head()

,Unnamed: 0,X,ID,STATUS,START_DATE,SNAPSHOT_MONTH,MoB
0,92833,92953,5008804,X,2021-09-01,2021-09-01,0
1,92834,92952,5008804,0,2021-09-01,2021-10-01,1
2,92835,92951,5008804,1,2021-09-01,2021-11-01,2
3,92836,92950,5008804,C,2021-09-01,2021-12-01,3
4,92837,92949,5008804,C,2021-09-01,2022-01-01,4


In [ ]:
filtered_by_start_date_df = credit_df[credit_df["START_DATE"] < "2021-06-01"].copy()

# 2. From this filtered DataFrame, identify records that define a BAD_2 customer
# Conditions: STATUS in ["1", "2", "3", "4", "5"] AND MoB < 5
bad_defining_records_df = filtered_by_start_date_df[
    (filtered_by_start_date_df["STATUS"].isin(["1", "2", "3", "4", "5"]))
    & (filtered_by_start_date_df["MoB"] < 5)
]

# 3. Get the unique IDs of customers who met these criteria
bad_2_customer_ids_updated = bad_defining_records_df["ID"].unique()

# 4. Create the 'BAD_2' column in the original credit_df
# Initialize all to 0 (not BAD)
credit_df["BAD_2"] = 0

# 5. Set 'BAD_2' to 1 for all records of customers who met the criteria
credit_df.loc[credit_df["ID"].isin(bad_2_customer_ids_updated), "BAD_2"] = 1

# Display the distribution of the new BAD_2 column in credit_df
print("BAD_2 column value counts in credit_df (updated logic):")
print(credit_df["BAD_2"].value_counts())

# Display the first few rows of credit_df with the new column
print("\ncredit_df head with BAD_2 column (updated logic):")
credit_df.head()

BAD_2 column value counts in credit_df (updated logic):
BAD_2
0    743982
1     33565
Name: count, dtype: int64

credit_df head with BAD_2 column (updated logic):


,Unnamed: 0,X,ID,STATUS,START_DATE,SNAPSHOT_MONTH,MoB,BAD_2
0,92833,92953,5008804,X,2021-09-01,2021-09-01,0,0
1,92834,92952,5008804,0,2021-09-01,2021-10-01,1,0
2,92835,92951,5008804,1,2021-09-01,2021-11-01,2,0
3,92836,92950,5008804,C,2021-09-01,2021-12-01,3,0
4,92837,92949,5008804,C,2021-09-01,2022-01-01,4,0


In [ ]:
credit_df.groupby("START_DATE")["BAD_2"].sum().reset_index()

,START_DATE,BAD_2
0,2017-12-01,559
1,2018-01-01,630
2,2018-02-01,436
3,2018-03-01,729
4,2018-04-01,868
...,...,...
56,2022-08-01,0
57,2022-09-01,0
58,2022-10-01,0
59,2022-11-01,0
